# GPU Setting

In [1]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
USE_GPU = True

if USE_GPU and tf.config.list_physical_devices('GPU'):
    device = 'GPU'
    print("Using GPU")
else:
    device = 'CPU'
    print("Using CPU")

2024-11-20 06:11:38.924028: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-20 06:11:38.988289: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/asko/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Num GPUs Available:  1
Using GPU


2024-11-20 06:11:39.957470: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-20 06:11:39.979017: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-20 06:11:39.979167: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


# Loading Dataset

In [2]:
import tarfile
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import random

# Path to the downloaded tgz file
tgz_path = "/home/asko/Documents/workspace/Fall-24/682/project/dataset/imagenette2.tgz"
extract_path = "./imagenette"  # Target folder for extraction

# Extract the file
with tarfile.open(tgz_path, "r:gz") as tar:
    tar.extractall(path=extract_path)
print("Extraction completed.")


transform = transforms.Compose([
    transforms.Resize((224,224)),  # Resize images to a size suitable for VGG16
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize as per VGG16
])
extract_path_train = "./imagenette/imagenette2/train"
train_dataset = datasets.ImageFolder(
    root=extract_path_train,  # Imagenette URL
    transform=transform
)
extract_path_val = "./imagenette/imagenette2/val"
val_dataset = datasets.ImageFolder(
    root=extract_path_val,  # Imagenette URL
    transform=transform
)

batch_size = 8
sampler_train = sampler.SubsetRandomSampler(range(len(train_dataset)))
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler_train)

batch_size = 8
sampler_val = sampler.SubsetRandomSampler(range(1000))
val_loader = DataLoader(val_dataset, batch_size=batch_size, sampler=sampler_val)

# batch_size = 16
# sampler_test = sampler.SubsetRandomSampler(range(1000, len(val_dataset)))
# test_loader = DataLoader(val_dataset, batch_size=batch_size, sampler=sampler_test)


/home/asko/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Extraction completed.


In [13]:
import torch

torch.cuda.set_per_process_memory_fraction(1., device=0) 

In [3]:
import tensorflow as tf
from student.StudentCNN import StudentCNN
import torch

alpha = 0.5
temperature = 3

# Load the VGG16 Teacher model
import dill
with open("ta_model_retrained.pkl", "rb") as f:
    ta_model = dill.load(f)

# Load the VGG16 model
student_model = StudentCNN(alpha=alpha, temperature=temperature, num_of_classes=10)

device = 'cuda' if len(tf.config.list_physical_devices('GPU'))!=0 else 'cpu'
ta_model.to(device)
student_model.to(device)
print(device)



cuda


In [4]:
ta_outputs = []

for batch_idx, (images, _) in enumerate(train_loader):
    images = images.to(device, dtype=torch.float32)
    torch.save(ta_model(images), f"outputs/TA_VGG/ta_outputs_{batch_idx}.pt")
    # teacher_outputs.append(teacher_model(images))

In [5]:
def check_accuracy_part34(loader, model):
    print('Checking accuracy on validation set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=torch.float32)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))


In [7]:
import torch.optim as optim
import torch

optimizer = optim.Adam(student_model.parameters(), lr=0.01)
# optimizer = optim.SGD(student_model.parameters(), lr=0.01, weight_decay=1e-4, momentum=0.9, nesterov=True)
torch.set_grad_enabled(True)

num_epochs = 40
# batch_size = 16  
model_weights = None
print_every = 100

for epoch in range(num_epochs): 
    for batch_idx, (images, labels) in enumerate(train_loader):
        student_model.train()           
        # print('h1')
        images = images.to(device, dtype=torch.float32)
        labels = labels.to(device, dtype=torch.long)
        # print('h2')
        student_output = student_model(images)
        ta_output = torch.load(f"outputs/TA_VGG/ta_outputs_{batch_idx}.pt")
        # print('h3')
        loss = student_model.risk(Y=labels, ta_preds=ta_output, output=student_output)
        # print('h4')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
        if batch_idx % print_every == 0:
            print('Iteration %d, loss = %.4f' % (batch_idx, loss.item()))
            check_accuracy_part34(val_loader, student_model)
            print()
            
        torch.cuda.empty_cache()
        del ta_output, student_output, images, labels, loss
        
    print(f"Epoch {epoch} complete")

Iteration 0, loss = 1.3168
Checking accuracy on validation set
Got 80 / 1000 correct (8.00)

Iteration 100, loss = 1.0041
Checking accuracy on validation set
Got 36 / 1000 correct (3.60)

Iteration 200, loss = 1.3004
Checking accuracy on validation set
Got 214 / 1000 correct (21.40)

Iteration 300, loss = 1.2578
Checking accuracy on validation set
Got 129 / 1000 correct (12.90)

Iteration 400, loss = 1.2679
Checking accuracy on validation set
Got 362 / 1000 correct (36.20)

Iteration 500, loss = 1.4298
Checking accuracy on validation set
Got 146 / 1000 correct (14.60)

Iteration 600, loss = 1.1669
Checking accuracy on validation set
Got 209 / 1000 correct (20.90)

Iteration 700, loss = 1.1882
Checking accuracy on validation set
Got 357 / 1000 correct (35.70)

Iteration 800, loss = 1.3193
Checking accuracy on validation set
Got 185 / 1000 correct (18.50)

Iteration 900, loss = 1.3211
Checking accuracy on validation set
Got 256 / 1000 correct (25.60)

Iteration 1000, loss = 1.2807
Checki

In [ ]:
check_accuracy_part34(train_loader, model=student_model)

Checking accuracy on validation set
Got 9469 / 9469 correct (100.00)


In [10]:
import dill
with open("student_model_retrained.pkl", "wb") as f:
    dill.dump(student_model, f)

In [9]:
batch_size = 16
sampler_test = sampler.SubsetRandomSampler(range(1000, len(val_dataset)))
test_loader = DataLoader(val_dataset, batch_size=batch_size, sampler=sampler_test)

check_accuracy_part34(test_loader, student_model)

Checking accuracy on validation set
Got 1978 / 2925 correct (67.62)
